### Submission Information:
- Name: Vũ Thị Phương Anh
- StudentID: K204110558
- Course: BIG DATA AND ITS APPLICATIONS
- Instructor: TS. Nguyễn Thôn Dã
- Assignment: Twitter API

## Import libraries
> tweepy

> tdqm | !pip install tdqm

> pandas

In [3]:
!pip install tdqm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1319 sha256=e5fe4a21448171a667e277249db00d8acb47024c3e2780d400b517c9c11f35cb
  Stored in directory: /root/.cache/pip/wheels/c6/f0/d9/9fa5ff78c0f9d5a0a427bbbb4893c283520ddfccb885ea2205
Successfully built tdqm


In [1]:
# import tweepy
import tweepy as tw
from tqdm import tqdm
import pandas as pd

## Part 1: Twitter API credentials
> API Key: EfPx4qIYloWkWTLn5aaFvshde

> API Secret: 53PC4H1Sq4JohEADkaOD5uo3XiK6F7wXSFm4yQ8NzOe0Rw734m

In [2]:
# your Twitter API key and API secret
my_api_key = "EfPx4qIYloWkWTLn5aaFvshde"
my_api_secret = "53PC4H1Sq4JohEADkaOD5uo3XiK6F7wXSFm4yQ8NzOe0Rw734m"
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

## Part 2: Collect the tweets

### 2.1. Set up search query
> Can change the keyword, itemQuantity, language, and filter

- keyword: covid19, president trump, etc.
- itemQuantity: 100, 1000, 10000, etc.
- language: 'vi','en','zhCN', 'de', 'sv', 'ru', 'ko', 'it', 'fr', 'es', 'th', 'ja', 'pl', 'nl'
- filter: 'retweets', 'images', 'videos', 'links', 'replies'

In [4]:
# keywords to search
keyword = "covid-19"
# number of tweets to search
itemQuantity = 50
# language
language = "en"
# search query
search_query = f"#{keyword} -filter:retweets"

### 2.2. Collect

In [5]:
# get tweets from the API
# search for tweets in last year
tweets = tw.Cursor(api.search, 
                    q=search_query, 
                    lang=language, 
                    since="2020-01-01", 
                    tweet_mode="extended").items(itemQuantity)

# store the API responses in a list
tweets_copy = []
for tweet in tqdm(tweets, desc='Appending', unit=' tweets', total=itemQuantity):
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Appending: 100%|██████████| 50/50 [00:00<00:00, 52.35 tweets/s]

Total Tweets fetched: 50


## Part 3: Save the tweets into DataFrame

### 3.1. Save the tweets into DataFrame

In [6]:
# intialize the dataframe
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in tqdm(tweets_copy, desc='Saving', unit=' tweets', total=len(tweets_copy)):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = pd.concat([tweets_df, pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source})], ignore_index=True)

    tweets_df = tweets_df.reset_index(drop=True)

Saving: 100%|██████████| 50/50 [00:06<00:00,  7.67 tweets/s]


### 3.2. Show DataFrame

In [7]:
# show the dataframe
tweets_df.head(10)

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,zene,the art of hieu,•madness in its sweetest form•\n19 • ENTJ-T• #...,False,2022-11-06 10:36:26,@katloredoo gay porn premium discord nitro wit...,None,Twitter for Android
1,Michelle on KP 💉💉💉💉😷,"Lakebay, WA, USA",Childfree. Socialist. Cat lover. Antiwar. Pro ...,False,2022-11-06 10:36:05,"""If someone had a lab-confirmed case of COVID-...",None,Twitter for Android
2,JolyDeschanel,,,False,2022-11-06 10:35:44,Not true. https://t.co/wwmR1gUmzv https://t.co...,None,Twitter Web App
3,RT_India,India,,False,2022-11-06 10:35:26,❗️Omicron Subvariant Surges: 🇨🇳 Infections Hit...,None,Twitter Web App
4,New Scientist,Worldwide,The best place to find out what’s new in scien...,True,2022-11-06 10:35:04,"A soup of omicron subvariants, primarily BQ.1....",None,Echobox
5,Hannibal Lupe,Denmark,,False,2022-11-06 10:34:39,@StevenHailAus @AlanKohler Yeah... \n- \nAnd T...,None,Twitter Web App
6,gal Matheys,,not American (yet) but full blown American mag...,False,2022-11-06 10:34:37,Who can flip the covid vax mandate for non US ...,None,Twitter Web App
7,John,,"report: N Ireland cases: 712,190 plus. victims...",False,2022-11-06 10:34:36,Cancer Drug Imatinib Effective Treating COVID-...,None,Twitter Web App
8,Healthwatch Leicester and Leicestershire,"Leicester, England",Your spotlight on health and social care in Le...,False,2022-11-06 10:34:30,COVID-19 and flu spread more quickly in winter...,"[vaccine, booster, GetBoosted]",recurpost.com
9,John,,"report: N Ireland cases: 712,190 plus. victims...",False,2022-11-06 10:34:12,NIH researchers identify potential new antivir...,"[Balnamore, Ballymoney, Seacon, Carnany]",Twitter Web App


### 3.3. Statistics of the tweets DataFrame

In [8]:
tweets_df.describe(include='all', datetime_is_numeric=True)

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
count,50,50,50,50,50,50,13,50
unique,46,35,40,2,NaN,50,12,13
top,John,,,False,NaN,@katloredoo gay porn premium discord nitro wit...,[news],Twitter Web App
freq,4,14,7,46,NaN,1,2,19
mean,NaN,NaN,NaN,NaN,2022-11-06 10:29:54.400000,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,2022-11-06 10:23:51,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2022-11-06 10:26:29.500000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,2022-11-06 10:30:24.500000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,2022-11-06 10:33:09.500000,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,2022-11-06 10:36:26,NaN,NaN,NaN
